In [10]:
import torch
from torch.utils.data import Dataset
import h5py
import numpy as np
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.nn as nn
from torchvision.models import resnet50, ResNet50_Weights
from tqdm import tqdm
import matplotlib.pyplot as plt
from collections import Counter

# 1. データ整合性チェック関数
def check_data_leak(train_path, val_path, test_path):
    """データセット間の重複をチェック"""
    with h5py.File(train_path, 'r') as train, \
         h5py.File(val_path, 'r') as val, \
         h5py.File(test_path, 'r') as test:
        
        train_data = train['x'][:]
        val_data = val['x'][:]
        test_data = test['x'][:]
        
        # メモリ効率的なハッシュ比較
        train_hashes = {hash(x.tobytes()) for x in train_data}
        val_hashes = {hash(x.tobytes()) for x in val_data}
        test_hashes = {hash(x.tobytes()) for x in test_data}
        
        print(f"Train-Val 重複: {len(train_hashes & val_hashes)}件")
        print(f"Train-Test 重複: {len(train_hashes & test_hashes)}件")

# データリークチェック実行
check_data_leak('camelyonpatch_level_2_split_train_x.h5',
               'valid_x_uncompressed.h5',
               'camelyonpatch_level_2_split_test_x.h5')

# 2. データ拡張と正規化
mean = [0.702, 0.538, 0.691]
std = [0.238, 0.279, 0.213]

transform_test = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

# 3. 癌特異的データ拡張
class CancerSpecificAugment:
    def __call__(self, img, label):
        if label == 1:  # 癌クラスのみに適用
            img = transforms.functional.adjust_sharpness(img, 1.8)  # 強度を少し下げる
            img = transforms.functional.adjust_contrast(img, 1.1)
        return img

# 4. カスタムデータセット
class PCamDataset(Dataset):
    def __init__(self, h5_x_path, h5_y_path, transform=None):
        self.x_h5 = h5py.File(h5_x_path, 'r')['x']
        self.y_h5 = h5py.File(h5_y_path, 'r')['y']
        self.transform = transform
        self.augment = CancerSpecificAugment()
        
        # ラベルデータをNumPy配列に変換してからbincountを適用
        y_data = np.array(self.y_h5[:])
        self.class_counts = np.bincount(y_data.flatten())  # flatten()で1次元化
        
        print(f"Class distribution - 0: {self.class_counts[0]}, 1: {self.class_counts[1]}")

    def __len__(self):
        return len(self.y_h5)

    def __getitem__(self, idx):
        image = self.x_h5[idx]
        label = self.y_h5[idx].item()  # .item()でPythonスカラーに変換
        
        if self.transform:
            image = self.transform(image)
            image = self.augment(image, label)
            
        return image, label

# 5. 改良版Focal Loss
class BalancedFocalLoss(nn.Module):
    def __init__(self, alpha=0.75, gamma=2.0, smoothing=0.1):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.smoothing = smoothing

    def forward(self, inputs, targets):
        # ラベル平滑化
        targets = (1 - self.smoothing) * targets + self.smoothing / inputs.size(1)
        
        bce_loss = nn.functional.binary_cross_entropy_with_logits(
            inputs, targets, reduction='none'
        )
        pt = torch.exp(-bce_loss)
        loss = (self.alpha * (1-pt)**self.gamma * bce_loss).mean()
        return loss

# 6. データセット初期化
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.3),
    transforms.RandomAffine(degrees=10, translate=(0.1, 0.1)),
    transforms.ColorJitter(brightness=0.05, contrast=0.05),
    transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 1.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
    transforms.RandomErasing(p=0.1, scale=(0.02, 0.1))
])

train_dataset = PCamDataset('camelyonpatch_level_2_split_train_x.h5',
                          'camelyonpatch_level_2_split_train_y.h5',
                          transform=train_transform)

val_dataset = PCamDataset('valid_x_uncompressed.h5',
                         'valid_y_uncompressed.h5',
                         transform=transform_test)

test_dataset = PCamDataset('camelyonpatch_level_2_split_test_x.h5',
                         'camelyonpatch_level_2_split_test_y.h5',
                         transform=transform_test)

# 7. データローダー設定
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, 
                         shuffle=True, num_workers=4, pin_memory=True,
                         drop_last=True)  # 最後の不完全なバッチを除外

val_loader = DataLoader(val_dataset, batch_size=batch_size,
                       shuffle=False, num_workers=4, pin_memory=True)

test_loader = DataLoader(test_dataset, batch_size=batch_size,
                        shuffle=False, num_workers=4, pin_memory=True)

# 8. モデル構築
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# クラス重みの計算 (クラス不均衡対策)
class_weights = torch.tensor([
    1.0,  # Class 0
    train_dataset.class_counts[0] / train_dataset.class_counts[1]  # Class 1
], dtype=torch.float32).to(device)

model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)

# 事前学習層を凍結
for param in model.parameters():
    param.requires_grad = False

# 最終層の再設計
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 512),
    nn.BatchNorm1d(512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 256),
    nn.BatchNorm1d(256),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(256, 2)
)

# 勾配計算を有効にする層
for param in model.fc.parameters():
    param.requires_grad = True

model = model.to(device)

# 9. 損失関数と最適化
criterion = BalancedFocalLoss(alpha=0.8, gamma=2.0, smoothing=0.05)

# 層別学習率
optimizer = torch.optim.AdamW([
    {'params': model.fc.parameters(), 'lr': 1e-4},
    {'params': model.layer4.parameters(), 'lr': 5e-5}
], weight_decay=1e-4)

# 学習率スケジューラ
scheduler = torch.optim.lr_scheduler.CyclicLR(
    optimizer,
    base_lr=1e-5,
    max_lr=1e-4,
    step_size_up=2000,
    cycle_momentum=False
)

# 10. 訓練ループ
best_val_acc = 0.0
patience = 5
no_improve = 0
grad_scaler = torch.cuda.amp.GradScaler()

for epoch in range(30):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0
    
    pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/30')
    for imgs, labels in pbar:
        imgs, labels = imgs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        # 混合精度訓練
        with torch.cuda.amp.autocast():
            outputs = model(imgs)
            loss = criterion(outputs, labels)
        
        grad_scaler.scale(loss).backward()
        grad_scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        grad_scaler.step(optimizer)
        grad_scaler.update()
        
        # メトリクス計算
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        pbar.set_postfix({
            'Loss': f"{train_loss/(pbar.n+1):.4f}",
            'Acc': f"{100.*correct/total:.2f}%",
            'LR': f"{optimizer.param_groups[0]['lr']:.1e}"
        })
    
    # 検証フェーズ
    model.eval()
    val_correct = 0
    val_total = 0
    val_loss = 0.0
    
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            val_total += labels.size(0)
            val_correct += predicted.eq(labels).sum().item()
    
    val_acc = 100. * val_correct / val_total
    val_loss /= len(val_loader)
    print(f'\nValidation: Acc={val_acc:.2f}%, Loss={val_loss:.4f}')
    
    # ベストモデル保存
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), 'best_model.pth')
        no_improve = 0
        print(f"New best model saved! (Accuracy: {val_acc:.2f}%)")
    else:
        no_improve += 1
        if no_improve >= patience:
            print(f"\nEarly stopping at epoch {epoch+1}")
            break

# 11. 最終評価
model.load_state_dict(torch.load('best_model.pth', weights_only=True))
model.eval()

# テストセット評価
test_correct = 0
test_total = 0
all_preds = []
all_labels = []

with torch.no_grad():
    for imgs, labels in test_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(imgs)
        _, predicted = outputs.max(1)
        test_total += labels.size(0)
        test_correct += predicted.eq(labels).sum().item()
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_acc = 100. * test_correct / test_total
print(f'\nFinal Test Accuracy: {test_acc:.2f}%')

# クラスごとの精度
from sklearn.metrics import classification_report
print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=['Class 0', 'Class 1']))

# 混同行列
from sklearn.metrics import confusion_matrix
import seaborn as sns

cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Class 0', 'Class 1'],
            yticklabels=['Class 0', 'Class 1'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

Train-Val 重複: 0件
Train-Test 重複: 33件
Class distribution - 0: 131072, 1: 131072
Class distribution - 0: 16399, 1: 16369
Class distribution - 0: 16391, 1: 16377
Using device: cuda


/tmp/ipykernel_221743/2782016007.py:195: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  grad_scaler = torch.cuda.amp.GradScaler()
Epoch 1/30:   0%|          | 0/4096 [00:00<?, ?it/s]/tmp/ipykernel_221743/2782016007.py:210: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Epoch 1/30:   0%|          | 0/4096 [00:00<?, ?it/s]


ValueError: Target size (torch.Size([64])) must be the same as input size (torch.Size([64, 2]))

Label: 0, Shape: torch.Size([3, 96, 96])
Pixel range: -3.244 to 1.138


NameError: name 'plt' is not defined